# ecCodes

The ECMWF ecCodes library provides an API for decoding/encoding GRIB (General Regularly-distributed Information in Binary form) (editions 1 & 2) and BUFR (editions 3 & 4) messages. The Python interface (“eccodes” package) offers nearly one-to-one bindings to the C API, using NumPy arrays for field extraction and manipulation

https://learning.ecmwf.int/mod/scorm/player.php?a=176&currentorg=Introduction_to_BUFR_decoding_with_ecCodes_ORG&scoid=452
# BUFR decoding with ecCodes (endcoding and decoding tool)
- ECCODES = GRIB DECODER + BUFR decoder + GTS headers decoder
- BUFR = Binary UNiversal Form for the Representation of meteorogical data
- BUFR is a flexible binary format. It is mainly used to encode in situ and satellite observations (e.g. temeprature, bariometric pressura and visiability at a weather station) or upper air sounding (with ballons). BUFR can also represent forecast data.
- It is maintained by the World Meteorogical Organization (WMO). It belongs to the category of table-driven code form, where the meaning of data elements is determined by referring to a set of tables that are kept and maintained separately from the message itself. The WMO Manual on Codes is the only authoratative source for the GRIB and BUFR codes and fully describes their coding standards.
- BUFR messages are composed of sections. The sections encode the metadata as well as the data itself. Some metadata is like the instruments used to measure the temperature, etc. It has 3 headers sections and 2 data sections:
  - Indicator (total length, edition number), Identification (master table, data catrehory, date, time), Optional local use (whatever the originating centre needs for internal purposes, procesing, arhciving)
  - data description (structure), binary data
  - and an end section.
- ecCodes uses key/value pairs. The key is a unique identifier for an item of data, and the value of it.
- in ecCodes, the data is decoded only if you set the unpack key to 1.
  - For that, in section 3 there are sequences of descriptors, that together with tables, can hel to unpack/decode the data in section 4.
    - There are 4 types of descriptors in the ormat F-X-Y: F=[0 element,1 replication,2 operator,3 sequence]
    - 
- BUFR decoding formula = (intValue + Reference)*10⁻scale = value
- In the data section (4), the data may be compressed or uncompressed:
- ecCodes provides a set of command line tools which allow you to inspect or change the BUFR messages within files. The most important are:
    - bufr_count - print total number of message in files: bufr_count -v syn*.bufr
    - bufr_dump - dumps the content in different formats: bfr_dump -p my.bufr, bufr_dump -js (json (javascript object notation) output) ...
    - bufr_compare - compares 2 bufr files
    - bufr_ls - list a summary of header info
    - bufr_-filter - applies a set of rules to each message (really powerful). You can use it to filter and obtain whatever information you want. You have to define the rules in a macro.

- Conclusions:
  - BUFR is a table-driven binary format that is mostly used to enconde in situ and satellite observations.
  - BUFR messages consist of several sections and contain metadata as well as data
  - BUFR data consist of different types of descriptors and associated data.
  - ecCodes provides a set of command line tools which allow you to handle BUFR messsages within files.

In [1]:
!pip install eccodes

## Reading a BUFR (Binary Universal Form for the Representation of meteorological data) File

Below, we read a BUFR message (e.g., a radiosonde observation) and extract core fields: station ID, latitude, longitude, and temperature observations. We use codes_bufr_new_from_samples to load a sample template and decode variables; then we iterate over subsets.

- First we create an account at ecmwf
- we generate an api key
- with that we can use both the official ecmwf-api-client and the community‐developed ecmwf-opendata packages. 

In [2]:
pip install ecmwf-api-client

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install ecmwf-opendata

Note: you may need to restart the kernel to use updated packages.


In [4]:
%%writefile ~/.ecmwfapirc
# API key file for ECMWF WebAPI
url: https://api.ecmwf.int/v1
key: 73560544b486cd3111472514dac7a62f
email: davidfreirefernandez1998@gmail.com

Writing .ecmwfapirc


In [5]:
cat .ecmwfapirc

# API key file for ECMWF WebAPI
url: https://api.ecmwf.int/v1
key: 73560544b486cd3111472514dac7a62f
email: davidfreirefernandez1998@gmail.com


# Retrieving BUFR via the ECMWF Web API (MARS) in Python
- Now we can retrieve this data
- ECMWF’s MARS (Meteorological Archival and Retrieval System) stores BUFR observation files in classes such as od (operational data), rd (reanalysis), or ea (ERA data) streams.
- Each BUFR message has metadata (e.g., class, stream, type, date, time, expver, levtype, etc.) that you specify in your request.

MARS is ECMWF's Meteorological Archival and Retrieval System. This documentation focuses on facilities for the retrieval of data from ECMWF's operational and other archives, including the online Fields Data Base (FDB).
https://confluence.ecmwf.int/display/UDOC/MARS+user+documentation

In [15]:
from ecmwfapi import ECMWFDataServer

server = ECMWFDataServer()

server.retrieve({
    'stream'    : "oper",
    'levelist'  : "1/2/3/5/7/10/20/30/50/70/100/150/200/250/300/400/500/600/700/775/850/925/1000",
    'levtype'   : "pl",
    'param'     : "130.128/131.128/132.128/157.128",
    'dataset'   : "era40",
    'step'      : "0",
    'time'      : "00/06/12/18",
    'date'      : "2002-08-01/to/2002-08-31",
    'type'      : "an",
    'class'     : "e4",
    'format' : 'bufr',
    'target'    : "era40_2002-08-01to2002-08-31_00061218.bufr"
})


2025-06-03 01:27:49 ECMWF API python library 1.6.5
2025-06-03 01:27:49 ECMWF API at https://api.ecmwf.int/v1
2025-06-03 01:27:50 Welcome David Freire Fernandez
2025-06-03 01:27:55 In case of problems, please check https://confluence.ecmwf.int/display/WEBAPI/Web+API+FAQ or contact servicedesk@ecmwf.int
2025-06-03 01:27:56 Request submitted
2025-06-03 01:27:56 Request id: 683e337ccf1d95a18451bab9
2025-06-03 01:27:56 Request is submitted
2025-06-03 01:27:58 mars - ERROR  - 20250602.232756 - undefined value : bufr for parameter FORMAT
2025-06-03 01:27:58 mars - ERROR  - 20250602.232756 - Values are : 
2025-06-03 01:27:58 PACKED or P
2025-06-03 01:27:58 UNPACKED or U
2025-06-03 01:27:58 GRIB EDITION 1 or GRIB1
2025-06-03 01:27:58 GRIB EDITION 2 or GRIB2
2025-06-03 01:27:58 NETCDF
2025-06-03 01:27:58 ODB
2025-06-03 01:27:58 ASCII
2025-06-03 01:27:58 mars - FATAL  - 20250602.232756 - No request


APIException: 'ecmwf.API error 1: Bad request'